Python으로 실습하는 병렬컴퓨팅
===================================================
## MPI Parallelization with Python 

##### Supercomputing Center, KISTI | Oh-Kyoung Kwon

## MPI Parallelization

### Data decomposition (domain decomposition)

* **Each core has its own data** 
  
  - Each core communicates necessary data, synchronously and asynchronously.
  - Calculation is automatically divided

    <img src = "figs/fig_1.jpg" height="200">


* **It fits for MPI parallelization**
* **Programmer should do everything: difficult but very flexible** 
  - Data decomposition, task allocation, data communication.
* **It is a basic approach for cluster and MPP systems.**

### MPI (Message Passing Interface)

<img src = "figs/fig_2.jpg" height="250">


- **프로세스(Process)와 프로세서(Processor)** 
  
  - MPI는 프로세스 기준으로 작업할당 
  - 프로세서 대 프로세스 = 일대일(1:1) 또는 일대다(1:n)
  

- **메시지 (= 데이터 + 봉투(Envelope))**

  - 어떤 프로세스가 보내는가 
  - 어디에 있는 데이터를 보내는가 
  - 어떤 데이터를 보내는가 얼마나 보내는가 
  - 어떤 프로세스가 받는가 
  - 어디에 저장할 것인가 
  - 얼마나 받을 준비를 해야 하는가

- **꼬리표(tag)** 
  
  - 메시지 매칭과 구분에 이용 
  - 순서대로 메시지 도착을 처리할 수 있음 


- **커뮤니케이터(Communicator)** 

  - 서로간에 통신이 허용되는 프로세스들의 집합 
  - 기본적으로 만들어지는 커뮤니케이터: MPI.COMM_WORLD


- **랭크(Rank)** 

  - 동일한 커뮤니케이터 내의 프로세스들을 식별하기 위한 식별자
  - **0부터 시작**


- **점대점 통신(Point to Point Communication)** 
  
  - 두 개 프로세스 사이의 통신 
  - 하나의 송신 프로세스에 하나의 수신 프로세스가 대응


- **집합통신(Collective Communication)** 

  - 동시에 여러 개의 프로세스가 통신에 참여 
  - 일대다, 다대일, 다대다 대응 가능 
  - 여러 번의 점대점 통신 사용을 하나의 집합통신으로 대체 
    - 오류의 가능성이 적다. 
    - 최적화 되어 일반적으로 빠르다.

### MPI4py - introduction

- MPI for Python provides Python bindings for the Message Passing Interface (MPI) standard, 
  
- It allows Python applications to exploit multiple processors on workstations, clusters and supercomputers.

- This package builds on the MPI specification and provides an object oriented interface resembling the MPI-2 C++ bindings. 

  - Convenient communication of any picklable Python object

    - point-to-point (send & receive)
    - collective (broadcast, scatter & gather, reductions)

- Fast communication of Python object exposing the Python buffer interface (NumPy arrays, builtin bytes/string/array objects)

  - point-to-point (blocking/nonbloking/persistent send & receive)
  - collective (broadcast, block/vector scatter & gather, reductions)

- Process groups and communication domains

  - Creation of new intra/inter communicators
  - Cartesian & graph topologies

- This package builds on the MPI specification and provides an object oriented interface resembling the MPI-2 C++ bindings. (cont.)

- Parallel input/output:

  - read & write
  - blocking/non-blocking & collective/noncollective
  - individual/shared file pointers & explicit offset


- Dynamic process management

  - spawn & spawn multiple
  - accept/connect
  - name publishing & lookup


- One-sided operations

  - remote memory access (put, get, accumulate)
  - passive target syncronization (start/complete & post/wait)
  - active target syncronization (lock & unlock)

### MPI4py - 초기화 / 실행

- **라이브러리 import**

  - from mpi4py import MPI 

- **MPI 초기화와 마무리할 필요 없음**

  - Importing mpi4py already triggers MPI_INIT()
  - MPI_Finalize() is called when all python processes exit 

- **필요한 변수 초기화하기**

  - `comm = MPI.COMM_WORLD`
  - `myrank = comm.Get_rank()`
  - `nproc = comm.Get_size()`

- **병렬실행 (np: number of processes)**

  - `mpirun -np python mycode.py`

### MPI4py - mpirun / mpiexec

- **MPI distributions normally come with an implementation-specific execution utility.** 

  - Executes program multiple times (SPMD parallel programming) 
  - Supports multiple nodes 
  - Integrates with batch queueing systems 
  - Some implementations use “mpiexec”
  

- **Examples**

  ```shell
    mpirun -n 4 python script.py # on a laptop 
    mpirun --host n01,n02,n03,n04 python script.py 
    mpirun --hostfile hosts.txt python script.py 
    mpirun python script.py # with batch queueing system
    mpiexec -n 4 python script.py # on a windows OS 

  ```




### MPI4py - Installation

- **Prerequisites**
  - Python 3.6 or above
  - An MPI implementation like MPICH or Open MPI built with shared/dynamic libraries. 


- **Miniconda 설치**
  - Miniconda 공식 홈페이지: [link](https://docs.anaconda.com/free/miniconda/) 
  
  
- **`mpi` 이름으로 `conda`를 이용해서 설치**
  ```shell
  conda create -n mpi python=3.11 numpy mpi4py matplotlib
  ```

- **`conda` 활성화**
  ```shell
  conda activate mpi 
  ```

- **필요한 라이브러리 설치**
  ```shell
  conda install numpy
  ```

- **작업 실행**
  ```shell
  mpirun -np 3 python [FILE_NAME].py
  ```

- **`conda` 비활성화**
  ```shell
  mpirun -np 3 python mpi.py
  ```

### MPI4py - Using the Comm class to define communicator

In [ ]:
%%writefile ./examples/mpi2.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
myrank = comm.Get_rank()
nproc = comm.Get_size()

print("Hello World! from process {0} of {1} \n".format(myrank, nproc))

In [ ]:
!mpirun -np 3 python ./examples/mpi2.py

### MPI4py - 점대점통신 1 (Point to point communications)

- 가장 기본이 되는 통신 함수 (“send” and “recv”) $\leftarrow$ 소문자로 시작

  - `comm.send(obj, dest, tag=0)` 
  - `comm.recv(source=MPI.ANY SOURCE, tag=MPI.ANY TAG, status=None)` 
  
  - `“tag”` can be used as a filter 
  - `“dest”` must be a rank in communicator 
  - `“source”` can be a rank or MPI.ANY SOURCE (wild card) 
  - `“status”` used to retrieve information about recv’d message 
  - These are blocking operations
  - Basic Message Passing Process

  <img src = "figs/fig_3.jpg" height="250">

### MPI4py - MPI Data Types

<img src = "figs/fig_4.jpg" height="300">

<img src = "figs/fig_5.jpg" height="200">


### MPI4py - 점대점통신 1 예제

In [ ]:
%%writefile ./examples/p2p.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    msg = "Hello, world"
    comm.send(msg, dest=1)
elif rank == 1:
    s = comm.recv()
    print ("rank %d: %s" % (rank, s))

In [ ]:
!mpirun -np 2 python ./examples/p2p.py

### MPI4py - 점대점통신 2

- **Communications of buffer-like objects (“Send” and “Recv”) $\leftarrow$ 대문자로 시작**
  
  - **Buffer-like objects**
    
    - `Numpy arrays (class'numpy.ndarray’)`  
    - A list or tuple with 2 or 3 elements (or 4 elements for the vector variants)
      - 2 elements: [data, MPI.DOUBLE], data (a Numpy array)
      - 3 elements: [data, n, MPI.DOUBLE]: data, n (buffer of the first n elements.)
  
  - `comm.Send(buf, dest, tag=0)` 
  - `comm.Recv(buf, source=MPI.ANY SOURCE, tag=MPI.ANY TAG, status=None)` 
    
  - Advantage of working with buffer-like objects 
    - Communication is fast (close to the speed of MPI communication in C)
    
  - **Disadvantage**
    
    - Need to be more explicit when it comes to handling the allocation of memory space. 
    - The memory of the receiving buffer needs to be allocated prior to the communication, and the size of the sending buffer should not exceed that of the receiving buffer. 
  

In [ ]:
%%writefile ./examples/p2p_2.py

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# master process
if rank == 0:
    data = np.arange(4.)
    # master process sends data to worker processes by
    # going through the ranks of all worker processes
    for i in range(1, size):
        comm.Send(data, dest=i, tag=i)
        print('Process {} sent data:'.format(rank), data)

# worker processes
else:
    # initialize the receiving buffer
    data = np.zeros(4)
    # receive data from master process
    comm.Recv(data, source=0, tag=rank)
    print('Process {} received data:'.format(rank), data)

In [ ]:
!mpirun -np 2 python3 ./examples/p2p_2.py

### MPI4py - 점대점통신 3

- **Synchronous communication (Blocking comm.)**
  
  - A synchronous communication does not complete until the message has been received
  - Analogue to the beep or okay-sheet of a fax

- **Asynchronous communication (Non-blocking comm.)**

  - An asynchronous communication completes as soon as the message is on the way.
  - A post card or email
  

  <img src = "figs/fig_6.jpg" height="200">

- Communiation mode
  
  <img src = "figs/fig_7.jpg" height="300">

#### Avoiding deadlock (or hung)

- **MPI_SEND & MPI_RECV : Blocking communication**
  
  - **MPI_SEND**: the call does not return control to the calling program or routine, until the buffer containing the data to be copied unto the receiving process can be safely overwritten (This insures that the message being sent is not "corrupted" before the sending is complete)
  
  - **MPI_RECV**: the call does not return control to the calling program until the data to be received has in fact been received.

   <img src = "figs/fig_8.jpg" height="200">


#### No deadlock

```python
#!--Exchange messages
if mpirank == 0 :
	comm.Send(a, 1, tag1)
	comm.Recv(b, 1, tag2)
elif mpirank == 1 : 
	comm.Recv(a, 0, tag1)
	comm.Send(b, 0, tag2)
```

<img src = "figs/fig_9.jpg" height="200">


#### Unconditional deadlock

```python
# !--Exchange messages
if mpirank == 0 :
	comm.Recv(b, 1, tag2)
	comm.Send(a, 1, tag1)

elif mpirank == 1 : 
	comm.Recv(a, 0, tag1)
	comm.Send(b, 0, tag2)

```

<img src = "figs/fig_10.jpg" height="200">

#### Ensuring a Program is Safe (No deadlock) 

- **Must work the same using `comm.Send` and `comm.Ssend`**
  
  - comm.Ssend is synchronous mode send:  it will always wait until the receive has been posted on the receiving end. Even if the message is small and can be buffered internally, it will still wait until the message has started to be received on the other side.
  

- **Strategies for avoiding deadlock**
  
  - pay attention to order of send/receive in communication operations
  - use synchronous or buffered mode communication
  - use comm.Sendrecv
  - use non-blocking communication

#### `comm.Sendrecv`

- **`Sendrecv(sendbuf, dest, sendtag=0, recvbuf=None, source=ANY_SOURCE, recvtag=ANY_TAG, status=None)`**

  - sendbuf (BufSpec) –
  - dest (int) –
  - sendtag (int) –
  - recvbuf (BufSpec) –
  - source (int) –
  - recvtag (int) –
  - status (Optional[Status]) –

#### sendrecv.py: using `comm.Sendrecv`

In [ ]:
%%writefile ./examples/sendrecv.py 

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

a = np.zeros(size, dtype = int)
b = np.zeros(size, dtype = int)
a[rank] = rank + 1
inext = rank + 1
iprev = rank - 1

if rank == 0 :
    iprev = size - 1
if rank == size - 1 :
    inext = 0

for i in range(size) :
    if rank == i :
        print('BEFORE : myrank={0}, A = {1}'.format(rank, a))

comm.Sendrecv(a, inext, 77, b, iprev, 77)

for i in range(size) :
    if rank == i :
        print('AFTER  : myrank={0}, B = {1}'.format(rank, b))

In [ ]:
!mpirun -np 2 python3 ./examples/sendrecv.py

### MPI4py - 점대점통신 4

#### Non-blocking communications

- **`request = comm.Isend(… )`**

  - request: request handle


- **`request = comm.Irecv(…)`**

  - request: request handle


- `request.Wait(status=None)`


- `MPI.Request.Waitall(requests, statuses=None)`



#### isendrecv.py: Example of Non-blocking communications

In [ ]:
%%writefile ./examples/isendrecv.py

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank() # myrank = comm.rank

data  = np.zeros(100, dtype = float)
value = np.zeros(100, dtype = float)
req_list = []

if rank == 0 :
    for i in range(100) :
        data[i] = i * 100
        req_send = comm.Isend(data[i:i+1], dest = 1, tag = i)
        req_list.append(req_send)
elif rank == 1 :
    for i in range(100) :
        req_recv = comm.Irecv(value[i:i+1] , source = 0, tag = i)
        req_list.append(req_recv)

MPI.Request.Waitall(req_list)

if rank == 0 :
    print("data[99] = {0}\n".format(data[99]))

if rank == 1 :
    print("value[99] = {0}\n".format(value[99]))

In [ ]:
!mpirun -np 2 python ./examples/isendrecv.py

**What is the difference between `comm.Send` and `comm.Isend?`**
  - *Send:* the call does not return control to the calling program or routine, until the buffer containing the data to be copied unto the receiving process can be safely overwritten

  - *Isend:* the call returns control to the calling routine immediately after posting the send call, before it is safe to overwrite (or use) the buffer being sent.

    $\rightarrow$ It is free from deadlock

**User should control the safe transfer of data by using `comm.Wait`**

  - Before the program is to use the sent/received buffer, a call to comm.Wait is necessary.
  
  - `comm.Wait` is a blocking routine. It does not return control to the calling routine until it is safe to re-use the buffer.

**Non-blocking communication makes a big room for computation**
  
  - This allows the program to proceed with computations not involving the communication buffer, while the communication completes.

```python
if rank == 0 :
    for i in range(100) :
        data[i] = i * 100
        req_send = comm.Isend(data[i:i+1], dest = 1, tag = i)
        req_list.append(req_send)
elif rank == 1 :
    for i in range(100) :
        req_recv = comm.Irecv(value[i:i+1] , source = 0, tag = i)
        req_list.append(req_recv)

MPI.Request.Waitall(req_list)
```

**$\rightarrow$ We can put calculation**

### MPI4py - 집합통신 (Collective communicator)

- **A group of processes participate in the communication** 

- **Based on Point to Point communication**

- **More efficient, better performance than P2P communications**

- **Special feature**

  - All processes in the communicator group must be called

  - No message tag
  

  <img src = "figs/fig_11.jpg" height="500">


#### `comm.bcast(sendobj, root=0)`
  <img src = "figs/fig_12.jpg" height="200">


In [ ]:
%%writefile ./examples/bcast.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.rank

if rank == 0:
    data = {'a': 1, 'b': 2, 'c': 3}
else:
    data = None 
    
data = comm.bcast(data, root=0)
print('rank', rank, data)



In [ ]:
!mpirun -np 4 python3 ./examples/bcast.py

#### `comm.reduce(sendobj, op=MPI.SUM, root=0)`
  <img src = "figs/fig_13.jpg" height="200">


In [ ]:
%%writefile ./examples/sum.py

from mpi4py import MPI
import numpy as np
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank ()
size = comm.Get_size ()

local = random.randint(2, 5)
print("rank: {}, local: {}".format(rank, local))

sum = comm.reduce(local, MPI.SUM, root=0)
if (rank==0):
    print ("sum: ", sum)

In [ ]:
!mpirun -np 4 python ./examples/sum.py

#### `comm.scan(sendobj, op=MPI.SUM, root=0)`
  <img src = "figs/fig_14.jpg" height="200">


In [ ]:
%%writefile ./examples/scan.py

from mpi4py import MPI
import numpy as np
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank ()
size = comm.Get_size ()

local = random.randint(2, 5)
print("rank: {}, local: {}".format(rank, local))

scan = comm.scan(local, MPI.SUM)
print ("rank:", rank, "sum: ", scan)

In [ ]:
!mpiexec -np 4 python ./examples/scan.py

#### `recvobj = comm.gather(sendobj, root=0)`
  <img src = "figs/fig_15.jpg" height="200">


In [ ]:
%%writefile ./examples/gather1.py

import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

recvbuf = comm.gather(rank)
if rank == 0:
    print('Gathered array: {}'.format(recvbuf))

In [ ]:
!mpirun -np 4 python ./examples/gather1.py

#### `comm.Gather(sendbuf, recvbuf, root=0)`
  <img src = "figs/fig_16.jpg" height="200">

In [ ]:
%%writefile ./examples/gather2.py

import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size ()

local_array = [rank] * 4
print("rank: {}, local_array: {}".format(rank, local_array))
sendbuf = np.array(local_array)

recvbuf = None
if rank == 0:
    recvbuf = np.empty(size*4, dtype=int)

comm.Gather(sendbuf=sendbuf, recvbuf=recvbuf)
if rank == 0:
    print("Gathered array: {}".format(recvbuf))

In [ ]:
!mpirun -np 4 python ./examples/gather2.py

#### `comm.Gatherv(sendbuf, recvbuf=(recvbuf, sendcounts), root=0)`
  <img src = "figs/fig_17.jpg" height="200">

In [ ]:
%%writefile ./examples/gatherv.py

import numpy as np
from mpi4py import MPI
import random

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
local_array = [rank] * random.randint(2, 5)
print("rank: {}, local_array: {}".format(rank, local_array))
sendbuf = np.array(local_array)
sendcounts = np.array(comm.gather(len(sendbuf), 0))
recvbuf = None
if rank == 0:
    print("sendcounts: {}, total: {}".format(sendcounts, sum(sendcounts)))
    recvbuf = np.empty(sum(sendcounts), dtype=int)
comm.Gatherv(sendbuf=sendbuf, recvbuf=(recvbuf, sendcounts), root=0)
if rank == 0:
    print("Gathered array: {}".format(recvbuf))

In [ ]:
!mpirun -np 4 python ./examples/gatherv.py

#### `recvobj = comm.scatter(sendobj, root=0)`
  <img src = "figs/fig_18.jpg" height="200">

In [ ]:
%%writefile ./examples/scatter.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    data = [(i+1)**2 for i in range(size)]
else:
    data = None

data = comm.scatter(data, root=0)
assert data == (rank+1)**2
print("rank: {}, scattered: {}".format(rank, data))

In [ ]:
!mpirun -np 4 python ./examples/scatter.py

#### `recvobj = comm.allreduce(sendobj)`
  <img src = "figs/fig_19.jpg" height="200">

In [ ]:
%%writefile ./examples/allreduce.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
data = rank**2
data = comm.allreduce(data)

print("On rank", rank,"data =", data)

In [ ]:
!mpirun -np 3 python ./examples/allreduce.py

#### `recvobj = comm.allgather(sendobj)`
  <img src = "figs/fig_20.jpg" height="200">

In [ ]:
%%writefile ./examples/allgather.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
data = rank**2
data = comm.allgather(data)

print("On rank", rank,"data =", data)

In [ ]:
!mpirun -np 4 python ./examples/allgather.py

#### `recvobj = comm.alltoall(sendobj)`
  <img src = "figs/fig_21.jpg" height="200">

In [ ]:
%%writefile ./examples/alltoall.py

from mpi4py import MPI
comm = MPI.COMM_WORLD

rank = comm.Get_rank()
data = range(comm.Get_size())
print("On rank", rank,"original data =", list(data))
data = comm.alltoall(data)

print("On rank", rank,"transposed data =", data)

In [ ]:
!mpirun -np 4 python ./examples/alltoall.py

  <img src = "figs/fig_22.jpg" height="200">

### MPI4py - Loop parallelization

- **Loop is frequently found in computer algorithm.** 

  <img src = "figs/fig_23.jpg" height="80">




- **Loop parallelizatioin is essential for most computational problem.**

  <img src = "figs/fig_24.jpg" height="80">


### MPI4py - Block distribution - para_range



  <img src = "figs/fig_25.jpg" height="80">

- **Suppose when you divide `n` by `p`, the quotient is `q` and the remainder is `r`.**
 
  - $n = p\times q + r$ 

- **Proccess `0, ..., r-1` are assigned `q+1` iterations each. The other processes are assigned `q` iterations.**

  - $n = r(q+1) + (p-r)q$ 


In [ ]:
## para_range functions 

def para_range(n1, n2, size, rank):
    iwork = divmod((n2 - n1 + 1), size)
    ista = rank * iwork[0] + n1 + min(rank, iwork[1])
    iend = ista + iwork[0] - 1
    if iwork[1] > rank :
        iend = iend + 1

    return ista, iend

def para_range2(n1, n2, size, rank):
  N=(n2-n1+1)//size+((n2-n1+1)%size > rank)
  end=comm.scan(N)
  start=end-N

  return n1+start-1, n1+end-1

### MPI4py - example1 (`map.py`)

In [ ]:
%%writefile ./examples/map.py

from mpi4py import MPI
import numpy as np
import math

comm = MPI.COMM_WORLD
rank = comm.Get_rank ()
size = comm.Get_size ()

x = range(20)
m = int(math.ceil(float(len(x)) / size))
x_chunk = x[rank*m:(rank+1)*m]
r_chunk = map(math.sqrt, x_chunk)
r = comm.reduce(list(r_chunk))
if (rank==0):
    print (r)

#serial => print(list(map(math.sqrt, x)))

In [ ]:
!mpiexec -np 4 python ./examples/map.py

### MPI4py - example2 (`pi.py`)

  <img src = "figs/fig_26.jpg" height="350">


In [ ]:
%%writefile ./examples/pi.py

import math
from time import perf_counter
#starting value of x
x=-1
dx=0.0000001
start_time = perf_counter()
iters=int((1-(-1))/dx)
#the sum of all the areas - start at 0
A=0.
for i in range(iters):
    A=A+math.sqrt(1-x**2)*dx
    x=x+dx
tpi=2*A
error = abs(tpi - math.pi)
end_time = perf_counter()
print ("pi is approximately %.16f, "
    "error is %.16f" % (tpi, error))
print('Elapsed wall clock time = %g seconds.' % (end_time-start_time) )
print(' ')

In [ ]:
!python3 ./examples/pi.py

In [ ]:
%%writefile ./examples/pi_mpi.py

from mpi4py import MPI
from time import perf_counter
import math
comm=MPI.COMM_WORLD
size=comm.Get_size()
rank=comm.Get_rank()
x=-1
dx=0.0000001
start_time = perf_counter()
iters=int((1-(-1))/dx)
N=iters//size+(iters % size > rank)
start=comm.scan(N)-N
A=0.
x=x+dx*start
for i in range(N):
    A=A+math.sqrt(1-x**2)*dx
    x=x+dx
##
A = comm.reduce(A, op=MPI.SUM, root=0) 
##

end_time = perf_counter()
if rank==0:
    tpi=2*A
    error = abs(tpi - math.pi)
    print ("pi is approximately %.16f, "
        "error is %.16f" % (tpi, error))
    print('Elapsed wall clock time = %g seconds.' % (end_time-start_time) )
    print(' ')

In [ ]:
!mpirun -np 4 python3 ./examples/pi_mpi.py


### MPI4py - example3: vector + vector (`vecadd.py`: serial)

In [ ]:
%%writefile ./examples/vecadd.py

from mpi4py import MPI
import numpy as np 

world_comm = MPI.COMM_WORLD
world_size = world_comm.Get_size()
my_rank = world_comm.Get_rank()
N = 10000000

# initialize a
start_time = MPI.Wtime()
a = np.ones( N )
end_time = MPI.Wtime()
if my_rank == 0:
    print("Initialize a time: " + str(end_time-start_time))

# initialize b
start_time = MPI.Wtime()
b = np.zeros( N )
for i in range( N ):
    b[i] = 1.0 + i
end_time = MPI.Wtime()
if my_rank == 0:
    print("Initialize b time: " + str(end_time-start_time))

# add the two arrays
start_time = MPI.Wtime()
for i in range( N ):
    a[i] = a[i] + b[i]
end_time = MPI.Wtime()
if my_rank == 0:
    print("Add arrays time: " + str(end_time-start_time))

# average the result
start_time = MPI.Wtime()
sum = 0.0

for i in range( N ):
    sum += a[i]
average = sum / N
end_time = MPI.Wtime()

if my_rank == 0:
    print("Average result time: " + str(end_time-start_time))
    print("Average: " + str(average))

In [ ]:
!python3 ./examples/vecadd.py

### MPI4py - example3: (`vecadd.py`: 점대점통신 함수를 이용한 병렬화)

In [ ]:
%%writefile ./examples/vecadd_mpi.py

from mpi4py import MPI
import numpy as np 

## fix me example3.1
## def para_range
def para_range(n1, n2, size, rank) :
    iwork = divmod((n2 - n1 + 1), size)
    ista = rank * iwork[0] + n1 + min(rank, iwork[1])
    iend = ista + iwork[0] - 1
    if iwork[1] > rank :
        iend = iend + 1
    return ista, iend

world_comm = MPI.COMM_WORLD
world_size = world_comm.Get_size()
my_rank = world_comm.Get_rank()
N = 10000000

## fix me example3.1
# mpi process(rank) 별로 workload 새로 구하기
(my_start, my_end) = para_range(0, N-1, world_size, my_rank)
workload = my_end - my_start + 1

# initialize a
start_time = MPI.Wtime()
a = np.ones(workload)
#FIX me example3.2 
end_time = MPI.Wtime()
if my_rank == 0:
    print("Initialize a time: " + str(end_time-start_time))

# initialize b
start_time = MPI.Wtime()
b = np.ones(workload) #FIX me example3.2 
for i in range(workload): #FIX me example3.3
    b[i] = 1.0 + i
end_time = MPI.Wtime()
if my_rank == 0:
    print("Initialize b time: " + str(end_time-start_time))

# add the two arrays
start_time = MPI.Wtime()
for i in range(workload): #FIX me example3.3
    a[i] = a[i] + b[i]
end_time = MPI.Wtime()
if my_rank == 0:
    print("Add arrays time: " + str(end_time-start_time))

# average the result
start_time = MPI.Wtime()
sum = 0.0

for i in range(workload): #FIX me example3.4
    sum += a[i]
#average = sum / N #FIX me example3.5
if my_rank == 0:
    world_sum = sum
    for i in range( 1, world_size ):
        sum_np = np.empty( 1 )
        world_comm.Recv( [sum_np, MPI.DOUBLE], source=i, tag=77 )
        world_sum += sum_np[0]
    average = world_sum / N
else:
    sum_np = np.array( [sum] )
    world_comm.Send( [sum_np, MPI.DOUBLE], dest=0, tag=77 )

end_time = MPI.Wtime()

if my_rank == 0:
    print("Average result time: " + str(end_time-start_time))
    print("Average: " + str(average))

- **example3.1: para_range를 이용해서 각 mpi process(rank) 별로 workload 새로 구하기** 

    ```python
        def para_range(n1, n2, size, rank) :
            iwork = divmod((n2 - n1 + 1), size)
            ista = rank * iwork[0] + n1 + min(rank, iwork[1])
            iend = ista + iwork[0] - 1
            if iwork[1] > rank :
                iend = iend + 1
            return ista, iend
    ```

    ```python
        # determine the workload of each rank
        (my_start, my_end) = para_range(0, N-1, world_size, my_rank)
        workload = my_end - my_start + 1
    ```

- **example3.2: a, b, vector 메모리 초기화 부분 수정**

    `np.ones(N)` $\rightarrow$ `np.one(workload)`

- **example3.3: b vector 초기화 / two vectors 더하기 계산 부분 각각 수정**

    `for i in range(N)` $\rightarrow$ `for i in range(workload)`

    `b[i] = 1.0 + i` $\rightarrow$ `b[i] = 1.0 + (i + my_start)`

- **example3.4: average the result 부분 수정**

    `for i in range(N)` $\rightarrow$ `for i in range(workload)`

- **example3.5: average 계산부분 수정**

    `average = sum / N` $\rightarrow$ 

    ```python
        if my_rank == 0:
            world_sum = sum
            for i in range( 1, world_size ):
                sum_np = np.empty( 1 )
                world_comm.Recv( [sum_np, MPI.DOUBLE], source=i, tag=77 )
                world_sum += sum_np[0]
            average = world_sum / N
        else:
            sum_np = np.array( [sum] )
            world_comm.Send( [sum_np, MPI.DOUBLE], dest=0, tag=77 )

    ```

In [ ]:
!mpirun -np 4 python3 ./examples/vecadd_mpi.py

### MPI4py - example3: (`vecadd.py`: 집합통신 함수를 이용한 병렬화)

In [ ]:
%%writefile ./examples/vecadd_mpi2.py

from mpi4py import MPI
import numpy as np 

world_comm = MPI.COMM_WORLD
world_size = world_comm.Get_size()
my_rank = world_comm.Get_rank()
N = 10000000

# initialize a
start_time = MPI.Wtime()
a = np.ones( N ) #FIX 
end_time = MPI.Wtime()
if my_rank == 0:
    print("Initialize a time: " + str(end_time-start_time))

# initialize b
start_time = MPI.Wtime()
b = np.zeros( N )
for i in range( N ): #FIX 
    b[i] = 1.0 + i
end_time = MPI.Wtime()
if my_rank == 0:
    print("Initialize b time: " + str(end_time-start_time))

# add the two arrays
start_time = MPI.Wtime()
for i in range( N ):
    a[i] = a[i] + b[i]
end_time = MPI.Wtime()
if my_rank == 0:
    print("Add arrays time: " + str(end_time-start_time))

# average the result
start_time = MPI.Wtime()
sum = 0.0

for i in range( N ): #FIX 
    sum += a[i]
average = sum / N #FIX 
end_time = MPI.Wtime()

if my_rank == 0:
    print("Average result time: " + str(end_time-start_time))
    print("Average: " + str(average))


- **average 계산부분 수정**
     
    ```python
        if my_rank == 0:
            world_sum = sum
            for i in range( 1, world_size ):
                sum_np = np.empty( 1 )
                world_comm.Recv( [sum_np, MPI.DOUBLE], source=i, tag=77 )
                world_sum += sum_np[0]
            average = world_sum / N
        else:
            sum_np = np.array( [sum] )
            world_comm.Send( [sum_np, MPI.DOUBLE], dest=0, tag=77 )

    ```

    $\rightarrow$

    ```python
        sum = np.array( [sum] )
        world_sum = np.zeros( 1 )
        world_comm.Reduce( [sum, MPI.DOUBLE], \
            [world_sum, MPI.DOUBLE], op = MPI.SUM, root = 0 )
        average = world_sum / N
    
    ```    

In [ ]:
!mpirun -np 4 python3 ./examples/vecadd_mpi2.py

### MPI4py - example4: Monte-Carlo (`mc.py`: serial)

In [ ]:
%%writefile ./examples/mc.py

from mpi4py import MPI
import numpy as np
#from cProfile import Profile
#from pstats import Stats

def generate_initial_state(method='random', file_name=None, num_particles=None, box_length=None):
    """
    Function generates initial coordinates for a LJ fluid simulation

    This function can generate coordintes either from a file (NIST LJ Fluid Configurations) or from
    a random configuration

    Parameters
    ----------

    method : str
        String the method to use to build the initial configuration for the LJ fluid simulation. Possible values are 'random'  or 'file' (Default value is 'random')
    file_name : str
        String of the the filename containing the initial starting coordinates. Only required when using the 'fille' method (Default value = None)
    num_particles : int
        Number of particules to use when populating the simualtion box with the 'random' method (Default value = None)
    box_length : float
        Size of one vertices of the simulation box. (Default value = None)

    Returns
    -------

    coordinates : np.array
        A (num_particles x 3) numpy array containing the coordinates of each LJ particle.

    Examples
    --------

    >>> generate_initial_state('random', num_particles = 1000, box_length = 20)
    array([[ 1.10202674,  4.24975121, -5.03322129],
        [ 9.13676284,  4.78807621, -8.26008762],
        [ 6.24720765, -7.17769567,  9.61620896],
        ...,
        [-3.47864571,  2.32867699, -1.31176807],
        [ 1.3302019 , -3.4160087 , -1.34698966],
        [ 0.56410479, -1.2309513 ,  4.71009776]])
    """


    if method == 'random':

        np.random.seed(seed=1)
        coordinates = (0.5 - np.random.rand(num_particles, 3)) * box_length

    elif method == 'file':

        coordinates = np.loadtxt(file_name, skiprows=2, usecols=(1,2,3))

    return coordinates


def lennard_jones_potential(rij2):
    """
    Function evaluates the unitless LJ potential given a squared distance

    Parameters
    ----------
    rij2 : float
        Distance squared between two particles

    Returns
    -------

    float
        Unitless LJ potential energy
    """
    # This function computes the LJ energy between two particles

    sig_by_r6 = np.power(1 / rij2, 3)
    sig_by_r12 = np.power(sig_by_r6, 2)
    return 4.0 * (sig_by_r12 - sig_by_r6)

def calculate_tail_correction(box_length, cutoff, number_particles):
    """
    This function computes the standard tail energy correction for the LJ potential

    Parameters
    ----------
    box_length : float/int
        length of simulation box
    cutoff: float/int
        the cutoff for the tail energy truncation
    num_particles: int
        number of particles

    Return
    ------
    e_correction: float
        tail correction of energy
    """


    volume = np.power(box_length, 3)
    sig_by_cutoff3 = np.power(1.0 / cutoff, 3)
    sig_by_cutoff9 = np.power(sig_by_cutoff3, 3)
    e_correction = sig_by_cutoff9 - 3.0 * sig_by_cutoff3

    e_correction *= 8.0 / 9.0 * np.pi * number_particles / volume * number_particles

    return e_correction

def minimum_image_distance(r_i, r_j, box_length):
    # This function computes the minimum image distance between two particles

    rij = r_i - r_j
    rij = rij - box_length * np.round(rij / box_length)
    rij2 = np.dot(rij, rij)
    return rij2

def get_particle_energy(coordinates, box_length, i_particle, cutoff2): 

    """
    This function computes the minimum image distance between two particles

    Parameters
    ----------
    r_i: list/array
        the potitional vection of the particle i
    r_j: list/array
        the potitional vection of the particle j
    box_length : float/int
        length of simulation box

    Return
    ------
    rij2: float
        the square of the shortest distance between the two particles and their images
    """


    e_total = 0.0

    i_position = coordinates[i_particle]

    particle_count = len(coordinates)

    for j_particle in range(particle_count):

        if i_particle != j_particle:

            j_position = coordinates[j_particle]

            rij2 = minimum_image_distance(i_position, j_position, box_length)

            if rij2 < cutoff2:
                e_pair = lennard_jones_potential(rij2)
                e_total += e_pair
    
    return e_total



def calculate_total_pair_energy(coordinates, box_length, cutoff2):
    e_total = 0.0
    particle_count = len(coordinates)

    for i_particle in range(particle_count):
        for j_particle in range(i_particle):

            r_i = coordinates[i_particle]
            r_j = coordinates[j_particle]
            rij2 = minimum_image_distance(r_i, r_j, box_length)
            if rij2 < cutoff2:
                e_pair = lennard_jones_potential(rij2)
                e_total += e_pair

    return e_total

def accept_or_reject(delta_e, beta):
    """Accept or reject a move based on the energy difference and system \
    temperature.

    This function uses a random numbers to adjust the acceptance criteria.

    Parameters
    ----------
    delta_e : float
        The difference between the proposed and current energies.
    beta : float
        The inverse value of the reduced temperature.

    Returns
    -------
    accept : booleen
        Either a "True" or "False" to determine whether to reject the trial.
    """
    # This function accepts or reject a move given the
    # energy difference and system temperature

    if delta_e < 0.0:
        accept = True

    else:
        random_number = np.random.rand(1)
        p_acc = np.exp(-beta * delta_e)

        if random_number < p_acc:
            accept = True
        else:
            accept = False

    return accept

def adjust_displacement(n_trials, n_accept, max_displacement):
    """Change the acceptance criteria to get the desired rate.

    When the acceptance rate is too high, the maximum displacement is adjusted \
     to be higher.
    When the acceptance rate is too low, the maximum displacement is \
     adjusted lower.

    Parameters
    ----------
    n_trials : integer
        The number of trials that have been performed when the function is \
         initiated.
    n_accept : integer
        The current number of accepted trials when the function is initiated.
    max_displacement : float
        The specified maximum value for the displacement of the trial.

    Returns
    -------
    max_displacement : float
        The adjusted displacement based on the acceptance rate.
    n_trials : integer, 0
        The new number of trials.
    n_accept : integer, 0
        The new number of trials.
    """
    acc_rate = float(n_accept) / float(n_trials)
    if (acc_rate < 0.38):
        max_displacement *= 0.8

    elif (acc_rate > 0.42):
        max_displacement *= 1.2

    n_trials = 0
    n_accept = 0

    return max_displacement, n_trials, n_accept


def main():
    start_simulation_time = MPI.Wtime()
    total_energy_time = 0.0
    total_decision_time = 0.0

    world_comm = MPI.COMM_WORLD
    world_size = world_comm.Get_size()
    my_rank = world_comm.Get_rank()

    #----------------
    # Parameter setup
    #----------------

    reduced_temperature = 0.9
    reduced_density = 0.9
    n_steps = 10000
    freq = 1000
    num_particles = 100
    simulation_cutoff = 3.0
    max_displacement = 0.1
    tune_displacement = True
    plot = True
    build_method = 'random'

    box_length = np.cbrt(num_particles / reduced_density)
    beta = 1.0 / reduced_temperature
    simulation_cutoff2 = np.power(simulation_cutoff, 2)
    n_trials = 0
    n_accept = 0
    energy_array = np.zeros(n_steps)

    #-----------------------
    # Monte Carlo simulation
    #-----------------------
    coordinates = generate_initial_state(method=build_method, \
        num_particles=num_particles, box_length=box_length)

    total_pair_energy = calculate_total_pair_energy(coordinates, box_length, simulation_cutoff2)
    tail_correction = calculate_tail_correction(box_length, simulation_cutoff, num_particles)

    n_trials = 0


    for i_step in range(n_steps):

        n_trials += 1

        i_particle = np.random.randint(num_particles)

        random_displacement = (2.0 * np.random.rand(3) - 1.0) * max_displacement

        start_energy_time = MPI.Wtime()
        current_energy = get_particle_energy(coordinates, box_length, i_particle, simulation_cutoff2)
        total_energy_time += MPI.Wtime() - start_energy_time

        proposed_coordinates = coordinates.copy()
        proposed_coordinates[i_particle] += random_displacement
        proposed_coordinates -= box_length * np.round(proposed_coordinates / box_length)

        start_energy_time = MPI.Wtime()
        proposed_energy = get_particle_energy(proposed_coordinates, box_length, \
            i_particle, simulation_cutoff2) 
        total_energy_time += MPI.Wtime() - start_energy_time

        start_decision_time = MPI.Wtime()

        delta_e = proposed_energy - current_energy

        accept = accept_or_reject(delta_e, beta)

        if accept:

            total_pair_energy += delta_e
            n_accept += 1
            coordinates[i_particle] += random_displacement

        total_energy = (total_pair_energy + tail_correction) / num_particles

        energy_array[i_step] = total_energy.item()

        if np.mod(i_step + 1, freq) == 0:
            if my_rank == 0:
                print(i_step + 1, energy_array[i_step])

            if tune_displacement:
                max_displacement, n_trials, n_accept = adjust_displacement(n_trials, n_accept, max_displacement)

        total_decision_time += MPI.Wtime() - start_decision_time

    if my_rank == 0:
        print("Total simulation time: " + str( MPI.Wtime() - start_simulation_time ) )
        print("    Energy time:       " + str( total_energy_time ) )
        print("    Decision time:     " + str( total_decision_time ) )

if __name__ == "__main__":
    # profiler = Profile()
    # profiler.run('main()')
    # stats = Stats(profiler)
    # stats.strip_dirs()
    # stats.sort_stats('cumulative')
    # stats.print_stats()
    main()

In [ ]:
!python3 ./examples/mc.py

### MPI4py - example4: Monte-Carlo (`mc_mpi1.py`: parallel 1)

- **example4.1: get_particle_energy 함수인자 부분 추가**
     
    ```python
        def get_particle_energy(coordinates, box_length, i_particle, cutoff2): 
    ```

    $\rightarrow$

    ```python
        def get_particle_energy(coordinates, box_length, i_particle, cutoff2, comm): 
    ```    

- **example4.2: main 함수 내 get_particle_energy 함수호출 부분 수정**
     
    ```python
        def main():
            ...
            current_energy = get_particle_energy(coordinates, box_length, i_particle, simulation_cutoff2)
            ...
            proposed_energy = get_particle_energy(proposed_coordinates, box_length, \
                i_particle, simulation_cutoff2)
            ...
    ```

    $\rightarrow$

    ```python
        def main():
            ...
            current_energy = get_particle_energy(coordinates, box_length, i_particle, simulation_cutoff2, world_comm)
            ...
            proposed_energy = get_particle_energy(proposed_coordinates, box_length, \
                i_particle, simulation_cutoff2, world_comm)
            ...
    ```    

- **example4.3: get_particle_energy 함수 내 loop parallelization **
     
    ```python
        def get_particle_energy(coordinates, box_length, i_particle, cutoff2, comm): 
            ...
            for j_particle in range(particle_count):
    ```

    $\rightarrow$

    ```python
        def get_particle_energy(coordinates, box_length, i_particle, cutoff2, comm): 
            ...
            my_rank = comm.Get_rank()
            world_size = comm.Get_size()
            for j_particle in range(my_rank, particle_count, world_size):
    ```    

- **example4.4: get_particle_energy 함수 내 loop parallelization을 위한 sum reduction **

    다음 loop 병렬화 분할 전략은 각 MPI rank가 world_size 간격으로 particles를 순회(iterate)합니다. 이때 첫 시작점(offset)은 my_rank입니다. 
    예를 들어, 4개 MPI rank로 실행할 경우, 랭크 0은 입자 0, 4, 8, 12 등을 순회하고, 랭크 1은 입자 1, 5, 9, 13 등을 순회합니다.
     
    ```python
        def get_particle_energy(coordinates, box_length, i_particle, cutoff2, comm): 
            ...
            for j_particle in range(my_rank, particle_count, world_size):
            ...
            return e_total
    ```

    $\rightarrow$

    ```python
        def get_particle_energy(coordinates, box_length, i_particle, cutoff2, comm): 
            ...
            ...
            for j_particle in range(my_rank, particle_count, world_size):
            ...
            
            # Sum the energy across all ranks
            e_single = np.array( [e_total] )
            e_summed = np.zeros( 1 )
            comm.Allreduce( [e_single, MPI.DOUBLE], [e_summed, MPI.DOUBLE], op = MPI.SUM )
            #comm.Reduce( [e_single, MPI.DOUBLE], [e_summed, MPI.DOUBLE], op = MPI.SUM, root = 0 )

            return e_summed[0]
    ```

In [ ]:
%%writefile ./examples/mc_mpi1.py

from mpi4py import MPI
import numpy as np
#from cProfile import Profile
#from pstats import Stats

def generate_initial_state(method='random', file_name=None, num_particles=None, box_length=None):
    """
    Function generates initial coordinates for a LJ fluid simulation

    This function can generate coordintes either from a file (NIST LJ Fluid Configurations) or from
    a random configuration

    Parameters
    ----------

    method : str
        String the method to use to build the initial configuration for the LJ fluid simulation. Possible values are 'random'  or 'file' (Default value is 'random')
    file_name : str
        String of the the filename containing the initial starting coordinates. Only required when using the 'fille' method (Default value = None)
    num_particles : int
        Number of particules to use when populating the simualtion box with the 'random' method (Default value = None)
    box_length : float
        Size of one vertices of the simulation box. (Default value = None)

    Returns
    -------

    coordinates : np.array
        A (num_particles x 3) numpy array containing the coordinates of each LJ particle.

    Examples
    --------

    >>> generate_initial_state('random', num_particles = 1000, box_length = 20)
    array([[ 1.10202674,  4.24975121, -5.03322129],
        [ 9.13676284,  4.78807621, -8.26008762],
        [ 6.24720765, -7.17769567,  9.61620896],
        ...,
        [-3.47864571,  2.32867699, -1.31176807],
        [ 1.3302019 , -3.4160087 , -1.34698966],
        [ 0.56410479, -1.2309513 ,  4.71009776]])
    """


    if method == 'random':

        np.random.seed(seed=1)
        coordinates = (0.5 - np.random.rand(num_particles, 3)) * box_length

    elif method == 'file':

        coordinates = np.loadtxt(file_name, skiprows=2, usecols=(1,2,3))

    return coordinates


def lennard_jones_potential(rij2):
    """
    Function evaluates the unitless LJ potential given a squared distance

    Parameters
    ----------
    rij2 : float
        Distance squared between two particles

    Returns
    -------

    float
        Unitless LJ potential energy
    """
    # This function computes the LJ energy between two particles

    sig_by_r6 = np.power(1 / rij2, 3)
    sig_by_r12 = np.power(sig_by_r6, 2)
    return 4.0 * (sig_by_r12 - sig_by_r6)

def calculate_tail_correction(box_length, cutoff, number_particles):
    """
    This function computes the standard tail energy correction for the LJ potential

    Parameters
    ----------
    box_length : float/int
        length of simulation box
    cutoff: float/int
        the cutoff for the tail energy truncation
    num_particles: int
        number of particles

    Return
    ------
    e_correction: float
        tail correction of energy
    """


    volume = np.power(box_length, 3)
    sig_by_cutoff3 = np.power(1.0 / cutoff, 3)
    sig_by_cutoff9 = np.power(sig_by_cutoff3, 3)
    e_correction = sig_by_cutoff9 - 3.0 * sig_by_cutoff3

    e_correction *= 8.0 / 9.0 * np.pi * number_particles / volume * number_particles

    return e_correction

def minimum_image_distance(r_i, r_j, box_length):
    # This function computes the minimum image distance between two particles

    rij = r_i - r_j
    rij = rij - box_length * np.round(rij / box_length)
    rij2 = np.dot(rij, rij)
    return rij2

def get_particle_energy(coordinates, box_length, i_particle, cutoff2): # FIX ME example4.1

    """
    This function computes the minimum image distance between two particles

    Parameters
    ----------
    r_i: list/array
        the potitional vection of the particle i
    r_j: list/array
        the potitional vection of the particle j
    box_length : float/int
        length of simulation box

    Return
    ------
    rij2: float
        the square of the shortest distance between the two particles and their images
    """


    e_total = 0.0

    i_position = coordinates[i_particle]

    particle_count = len(coordinates)

    # FIX me example4.3
    ##### get my rank  
    ##### get mpi size
    for j_particle in range(particle_count): # FIX ME example4.3

        if i_particle != j_particle:

            j_position = coordinates[j_particle]

            rij2 = minimum_image_distance(i_position, j_position, box_length)

            if rij2 < cutoff2:
                e_pair = lennard_jones_potential(rij2)
                e_total += e_pair
    
    # FIX me example4.4
    #################################
    # Sum the energy across all ranks
    return e_total
    #################################


def calculate_total_pair_energy(coordinates, box_length, cutoff2):
    e_total = 0.0
    particle_count = len(coordinates)

    for i_particle in range(particle_count):
        for j_particle in range(i_particle):

            r_i = coordinates[i_particle]
            r_j = coordinates[j_particle]
            rij2 = minimum_image_distance(r_i, r_j, box_length)
            if rij2 < cutoff2:
                e_pair = lennard_jones_potential(rij2)
                e_total += e_pair

    return e_total

def accept_or_reject(delta_e, beta):
    """Accept or reject a move based on the energy difference and system \
    temperature.

    This function uses a random numbers to adjust the acceptance criteria.

    Parameters
    ----------
    delta_e : float
        The difference between the proposed and current energies.
    beta : float
        The inverse value of the reduced temperature.

    Returns
    -------
    accept : booleen
        Either a "True" or "False" to determine whether to reject the trial.
    """
    # This function accepts or reject a move given the
    # energy difference and system temperature

    if delta_e < 0.0:
        accept = True

    else:
        random_number = np.random.rand(1)
        p_acc = np.exp(-beta * delta_e)

        if random_number < p_acc:
            accept = True
        else:
            accept = False

    return accept

def adjust_displacement(n_trials, n_accept, max_displacement):
    """Change the acceptance criteria to get the desired rate.

    When the acceptance rate is too high, the maximum displacement is adjusted \
     to be higher.
    When the acceptance rate is too low, the maximum displacement is \
     adjusted lower.

    Parameters
    ----------
    n_trials : integer
        The number of trials that have been performed when the function is \
         initiated.
    n_accept : integer
        The current number of accepted trials when the function is initiated.
    max_displacement : float
        The specified maximum value for the displacement of the trial.

    Returns
    -------
    max_displacement : float
        The adjusted displacement based on the acceptance rate.
    n_trials : integer, 0
        The new number of trials.
    n_accept : integer, 0
        The new number of trials.
    """
    acc_rate = float(n_accept) / float(n_trials)
    if (acc_rate < 0.38):
        max_displacement *= 0.8

    elif (acc_rate > 0.42):
        max_displacement *= 1.2

    n_trials = 0
    n_accept = 0

    return max_displacement, n_trials, n_accept




def main():
    start_simulation_time = MPI.Wtime()
    total_energy_time = 0.0
    total_decision_time = 0.0

    world_comm = MPI.COMM_WORLD
    world_size = world_comm.Get_size()
    my_rank = world_comm.Get_rank()

    #----------------
    # Parameter setup
    #----------------

    reduced_temperature = 0.9
    reduced_density = 0.9
    n_steps = 10000
    freq = 1000
    num_particles = 100
    simulation_cutoff = 3.0
    max_displacement = 0.1
    tune_displacement = True
    plot = True
    build_method = 'random'

    box_length = np.cbrt(num_particles / reduced_density)
    beta = 1.0 / reduced_temperature
    simulation_cutoff2 = np.power(simulation_cutoff, 2)
    n_trials = 0
    n_accept = 0
    energy_array = np.zeros(n_steps)

    #-----------------------
    # Monte Carlo simulation
    #-----------------------

    coordinates = generate_initial_state(method=build_method, \
        num_particles=num_particles, box_length=box_length)

    total_pair_energy = calculate_total_pair_energy(coordinates, box_length, simulation_cutoff2)
    tail_correction = calculate_tail_correction(box_length, simulation_cutoff, num_particles)

    n_trials = 0

    for i_step in range(n_steps):

        n_trials += 1

        i_particle = np.random.randint(num_particles)

        random_displacement = (2.0 * np.random.rand(3) - 1.0) * max_displacement

        start_energy_time = MPI.Wtime()
        current_energy = get_particle_energy(coordinates, box_length, i_particle, simulation_cutoff2) # FIX ME example4.2
        total_energy_time += MPI.Wtime() - start_energy_time

        proposed_coordinates = coordinates.copy()
        proposed_coordinates[i_particle] += random_displacement
        proposed_coordinates -= box_length * np.round(proposed_coordinates / box_length)

        start_energy_time = MPI.Wtime()
        proposed_energy = get_particle_energy(proposed_coordinates, box_length, \
            i_particle, simulation_cutoff2) # FIX ME example4.2
        total_energy_time += MPI.Wtime() - start_energy_time

        start_decision_time = MPI.Wtime()

        delta_e = proposed_energy - current_energy

        accept = accept_or_reject(delta_e, beta)

        if accept:

            total_pair_energy += delta_e
            n_accept += 1
            coordinates[i_particle] += random_displacement

        total_energy = (total_pair_energy + tail_correction) / num_particles

        energy_array[i_step] = total_energy.item()

        if np.mod(i_step + 1, freq) == 0:
            if my_rank == 0:
                print(i_step + 1, energy_array[i_step])

            if tune_displacement:
                max_displacement, n_trials, n_accept = adjust_displacement(n_trials, n_accept, max_displacement)

        total_decision_time += MPI.Wtime() - start_decision_time

    if my_rank == 0:
        print("Total simulation time: " + str( MPI.Wtime() - start_simulation_time ) )
        print("    Energy time:       " + str( total_energy_time ) )
        print("    Decision time:     " + str( total_decision_time ) )

if __name__ == "__main__":
    #profiler = Profile()
    #profiler.run('main()')
    #stats = Stats(profiler)
    #stats.strip_dirs()
    #stats.sort_stats('cumulative')
    #stats.print_stats()
    main()

In [ ]:
!mpirun -np 4 python3 ./examples/mc_mpi1.py
